In [29]:
import numpy as np
import cv2
import time
import os
import shutil
import dill
from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras.applications.densenet import DenseNet121
from keras.models import Sequential, load_model
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Activation
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, auc
from matplotlib import pyplot as plt
import plotly.graph_objects as go
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

# 1. Cut video to photos

In [16]:
def video_to_photos(input_file, output_dir):
    cap = cv2.VideoCapture(input_file)
    if cap.isOpened() == False:
        raise Exception('Video stream doesn\'t open!')
    frame_num = 1
    while cap.isOpened():
        ret, frame = cap.read()
        if ret == True:
            cv2.imwrite('{}/{}.jpg'.format(output_dir, frame_num), frame)
        else:
            break
        frame_num += 1
        print('\rProgress: {}'.format(frame_num), end='')
    cap.release()
    print()
    return frame_num

In [17]:
samples = ('train', 'valid', 'test')
classes = ('0', '1')
photos_cnt = dict()
for s in samples:
    for c in classes:
        input_file = 'videos/input/{}/{}/video.mp4'.format(s, c)
        output_dir = 'photos/{}/{}'.format(s, c)
        photos_cnt['{}/{}'.format(s, c)] = video_to_photos(input_file, output_dir)
        print('Stage "{}, {}" has done'.format(s, c))

Progress: 60422Stage "train, 0" has done
Progress: 32164Stage "train, 1" has done
Progress: 49990Stage "valid, 0" has done
Progress: 33042Stage "valid, 1" has done
Progress: 48774Stage "test, 0" has done
Progress: 31053Stage "test, 1" has done


In [22]:
print(photos_cnt['train/1'] / (photos_cnt['train/0'] + photos_cnt['train/1']))
print(photos_cnt['valid/1'] / (photos_cnt['valid/0'] + photos_cnt['valid/1']))
print(photos_cnt['test/1'] / (photos_cnt['test/0'] + photos_cnt['test/1']))

0.34739593459054285
0.39794296174968685
0.38900372054568005


In [2]:
def preprocessing_image(image):
    image /= 255
    return image

In [3]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
#     brightness_range=[0.2, 1.8],
    zoom_range=[0.15, 1.15],
    horizontal_flip=True,
    preprocessing_function=preprocessing_image
)

In [4]:
target_size = (224, 224)

In [5]:
train_datagen = datagen.flow_from_directory(
    'photos/train',
    batch_size=128,
    target_size=target_size,
    class_mode='binary',
    seed=1992
)
valid_datagen = datagen.flow_from_directory(
    'photos/valid',
    batch_size=128,
    target_size=target_size,
    class_mode='binary',
    seed=1992
)

Found 92584 images belonging to 2 classes.
Found 83030 images belonging to 2 classes.


In [6]:
densenet = DenseNet121(
    include_top=False,
    weights='imagenet',
    input_shape=(224,224,3)
)
densenet.trainable = False

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
model = Sequential()
model.add(densenet)
model.add(GlobalAveragePooling2D())
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Dropout(0.3))
model.add(Activation('sigmoid'))

model.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 7, 7, 1024)        7037504   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               262400    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
_____

In [8]:
model.compile(Adam(), loss='binary_crossentropy')

In [9]:
checkpoint = ModelCheckpoint(filepath='model/checkpoint_best_model.h5', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(patience=5, verbose=1)
reduce_lr = ReduceLROnPlateau(patience=2, factor=0.5)
# tensorboard = TensorBoard(log_dir='logs/tensorboard', histogram_freq=2, update_freq='batch')
callbacks = [checkpoint, early_stopping, reduce_lr]

In [10]:
history = model.fit_generator(
    generator=train_datagen,
    steps_per_epoch=int(np.ceil(train_datagen.samples / train_datagen.batch_size)),
    verbose=2,
    epochs=1000,
    callbacks=callbacks,
    validation_data=valid_datagen,
    validation_steps=int(np.ceil(valid_datagen.samples / valid_datagen.batch_size)),
    workers=8,
#     use_multiprocessing=True,
    class_weight={
        0: 1.0,
        1: 60422 / 32164
    }
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1000
 - 1133s - loss: 0.4807 - val_loss: 0.5367

Epoch 00001: val_loss improved from inf to 0.53672, saving model to model/checkpoint_best_model.h5
Epoch 2/1000
 - 1091s - loss: 0.3801 - val_loss: 0.6295

Epoch 00002: val_loss did not improve from 0.53672
Epoch 3/1000
 - 1103s - loss: 0.3636 - val_loss: 0.5982

Epoch 00003: val_loss did not improve from 0.53672
Epoch 4/1000
 - 1222s - loss: 0.3427 - val_loss: 0.6543

Epoch 00004: val_loss did not improve from 0.53672
Epoch 5/1000
 - 1108s - loss: 0.3365 - val_loss: 0.6622

Epoch 00005: val_loss did not improve from 0.53672
Epoch 6/1000
 - 1190s - loss: 0.3277 - val_loss: 0.5670

Epoch 00006: val_loss did not improve from 0.53672
Epoch 00006: early stopping


In [11]:
with open('model/history.pkl', 'wb') as f:
    dill.dump(history, f)

In [4]:
# with open('model/history.pkl', 'rb') as f:
#     history = dill.load(f)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


C:\Users\akoceruba\Anaconda3\envs\python3.6.6\lib\site-packages\keras\engine\saving.py:350: UserWarning:

Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.



In [12]:
train_loss = go.Scatter(
    x=list(range(1, len(history.history['loss']) + 1)),
    y=history.history['loss'],
    mode='lines+markers',
    name='Train loss',
    hoverinfo='y'
)
val_loss = go.Scatter(
    x=list(range(1, len(history.history['val_loss']) + 1)),
    y=history.history['val_loss'],
    mode='lines+markers',
    name='Validation loss',
    hoverinfo='y'
)

data = [train_loss, val_loss]
layout = go.Layout(
    title=dict(
        text='Learning curves'
    )
)
fig = go.Figure(data=data, layout=layout)

# fig.show()
pyo.plot(fig, filename='learning_curves.html')

'learning_curves.html'

In [7]:
model = load_model('model/checkpoint_best_model.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


C:\Users\akoceruba\Anaconda3\envs\python3.6.6\lib\site-packages\keras\engine\saving.py:350: UserWarning:

Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.



In [33]:
# def get_test_batch(test_dir, target_img_size, batch_size=256):
#     for class_name in os.listdir(test_dir):
#         test_dir_class = os.path.join(test_dir, class_name)
#         image_names = []
#         true_labels = []
#         batch = []
#         images = os.listdir(test_dir_class)
#         for i in range(len(images)):
#             test_dir_class_image = os.path.join(test_dir_class, images[i])
#             img = load_img(test_dir_class_image)
#             img = img_to_array(img)
#             img = cv2.resize(img, target_img_size)
#             img = preprocessing_image(img)
#             image_names.append(images[i][:-4])
#             true_labels.append(class_name)
#             batch.append(img)
#             if (i + 1) % batch_size == 0 or i == len(images) - 1:
#                 yield {
#                     'image_names': image_names,
#                     'true_labels': true_labels,
#                     'batch': np.array(batch)
#                 }
#                 image_names = []
#                 true_labels = []
#                 batch = []

# test_dir = 'photos/test/'
# get_test_batch_gen = get_test_batch(test_dir, target_size)

# image_names = []
# y_true = []
# y_pred = []
# for batch in tqdm_notebook(get_test_batch_gen):
#     image_names.extend(batch['image_names'])
#     y_true.extend(list(map(int, batch['true_labels'])))
#     y_pred.extend(list(model.predict_on_batch(batch['batch']).ravel()))
# predictions = [image_names, y_true, y_pred]

In [13]:
datagen_inference = ImageDataGenerator(
    preprocessing_function=preprocessing_image
)

In [14]:
def benchmark(func):
    import time
    def wrapper(*args, **kwargs):
        start_time = time.time()
        res = func(*args, **kwargs)
        end_time = time.time()
        delta = end_time - start_time
        minutes = int(delta / 60)
        seconds = int(delta - minutes * 60)
        print('Time of "{}": {} min. {} sec.'.format(func.__name__, minutes, seconds))
        return res
    return wrapper

In [15]:
@benchmark
def predict(model, datagen, directory, batch_size=256, target_size=(224, 224), workers=16):
    inference = datagen.flow_from_directory(
        directory,
        batch_size=batch_size,
        target_size=target_size,
        class_mode='binary',
        shuffle=False,
        seed=1992
    )
    image_names = list(map(lambda x: x[2:-4], inference.filenames))
    y_true = inference.classes
    y_pred = model.predict_generator(
        inference,
        steps=int(np.ceil(inference.samples / inference.batch_size)),
        workers=12,
        verbose=1
    ).ravel()
    return image_names, y_true, y_pred

In [16]:
image_names_train, y_train_true, y_train_pred = predict(model, datagen_inference, 'photos/train')

with open('predictions_train.pkl', 'wb') as f:
    dill.dump([image_names_train, y_train_true, y_train_pred], f)

image_names_valid, y_valid_true, y_valid_pred = predict(model, datagen_inference, 'photos/valid')

with open('predictions_valid.pkl', 'wb') as f:
    dill.dump([image_names_valid, y_valid_true, y_valid_pred], f)

image_names_test, y_test_true, y_test_pred = predict(model, datagen_inference, 'photos/test')

with open('predictions_test.pkl', 'wb') as f:
    dill.dump([image_names_test, y_test_true, y_test_pred], f)

Found 92584 images belonging to 2 classes.
362/362 [==============================] - ETA: 1:21:3 - ETA: 42:36  - ETA: 29:3 - ETA: 23:0 - ETA: 19:1 - ETA: 16:3 - ETA: 14:4 - ETA: 13:2 - ETA: 12:1 - ETA: 11:2 - ETA: 10:4 - ETA: 10:0 - ETA: 9:3 - ETA: 9: - ETA: 9: - ETA: 9: - ETA: 9: - ETA: 9: - ETA: 9: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 5: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 

312/312 [==============================] - ETA: 1:21:2 - ETA: 42:54  - ETA: 29:3 - ETA: 23:0 - ETA: 19:0 - ETA: 16:2 - ETA: 14:2 - ETA: 13:0 - ETA: 11:5 - ETA: 11:0 - ETA: 10:2 - ETA: 9:4 - ETA: 9: - ETA: 9: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - 

In [17]:
def gini_score(roc_auc):
    return 2 * roc_auc - 1

In [18]:
roc_auc = roc_auc_score(y_train_true, y_train_pred)
gini = gini_score(roc_auc)
display('ROC AUC and GINI:', roc_auc, gini)

'ROC AUC and GINI:'

0.9958172697269833

0.9916345394539665

In [19]:
roc_auc = roc_auc_score(y_valid_true, y_valid_pred)
gini = gini_score(roc_auc)
display('ROC AUC and GINI:', roc_auc, gini)

'ROC AUC and GINI:'

0.9224064716894416

0.8448129433788831

In [20]:
roc_auc = roc_auc_score(y_test_true, y_test_pred)
gini = gini_score(roc_auc)
display('ROC AUC and GINI:', roc_auc, gini)

'ROC AUC and GINI:'

0.8634312434458368

0.7268624868916735

In [22]:
def plot_curves(
    x_train,
    y_train,
    threshold_train,
    x_valid,
    y_valid,
    threshold_valid,
    x_test,
    y_test,
    threshold_test,
    curve_type='ROC',
    width=666,
    height=666
):
    
    trace_train = go.Scatter(
        x=x_train,
        y=y_train,
        mode='lines',
        name='Train',
        text=threshold_train
    )
    
    trace_valid = go.Scatter(
        x=x_valid,
        y=y_valid,
        mode='lines',
        name='Valid',
        text=threshold_valid
    )
    
    trace_test = go.Scatter(
        x=x_test,
        y=y_test,
        mode='lines',
        name='Test',
        text=threshold_test
    )

    data = [trace_train, trace_valid, trace_test]
    
    if curve_type == 'ROC':
        
        x_title = 'FPR'
        y_title = 'TPR'
        title = 'ROC Curves'
        
        trace_dot = go.Scatter(
            x=[0, 1],
            y=[0, 1],
            mode='lines',
            showlegend=False,
            line=dict(
                dash='dot'
            )
        )
        
        data.append(trace_dot)
        
    elif curve_type == 'PR':
        
        x_title = 'Recall'
        y_title = 'Precision'
        title = 'PR Curves'
    
    layout = go.Layout(
        title=dict(
            text=title
        ),
        hovermode='closest',
        width=width,
        height=height,
        xaxis=dict(
            title=dict(
                text=x_title
            )
        ),
        yaxis=dict(
            title=dict(
                text=y_title
            )
        )
    )
    fig = go.Figure(data=data, layout=layout)

#     fig.show()
    pyo.plot(fig, filename='{}_curves.html'.format(curve_type))

In [26]:
fpr_train, tpr_train, roc_thresholds_train = roc_curve(y_train_true, y_train_pred)
fpr_valid, tpr_valid, roc_thresholds_valid = roc_curve(y_valid_true, y_valid_pred)
fpr_test, tpr_test, roc_thresholds_test = roc_curve(y_test_true, y_test_pred)
plot_curves(
    fpr_train,
    tpr_train,
    roc_thresholds_train,
    fpr_valid,
    tpr_valid,
    roc_thresholds_valid,
    fpr_test,
    tpr_test,
    roc_thresholds_test
)

In [31]:
recall_train, precision_train, pr_thresholds_train = precision_recall_curve(y_train_true, y_train_pred)
recall_valid, precision_valid, pr_thresholds_valid = precision_recall_curve(y_valid_true, y_valid_pred)
recall_test, precision_test, pr_thresholds_test = precision_recall_curve(y_test_true, y_test_pred)
plot_curves(
    recall_train,
    precision_train,
    pr_thresholds_train,
    recall_valid,
    precision_valid,
    pr_thresholds_valid,
    recall_test,
    precision_test,
    pr_thresholds_test,
    curve_type='PR'
)

In [36]:
print('PR Train:', auc(recall_train, precision_train, reorder=True))
print('PR Valid:', auc(recall_valid, precision_valid, reorder=True))
print('PR Test:', auc(recall_test, precision_test, reorder=True))

PR Train: 0.5401038102068326
PR Valid: 0.5017587425002399
PR Test: 0.40754415134797234
